##### 문제 2-1 : 콤마 구분 리스트 파서 활용
* 사용자가 관심있는 분야(예: "음식", "스포츠", "영화" 등)를 입력하면, 해당 분야와 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분된 리스트로 출력하는 프로그램을 작성하세요.

In [66]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv
from pprint import pprint

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
        template=
        """
        {subject}와 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분된 리스트로 나열해주세요.
        각 항목은 특정 장소나 활동이나 음식을 나타내는 구체적인 이름이어야 합니다.
        
        예시:
        입력: "음식"
        출력: 명동 칼국수, 부산 돼지국밥 ...

        입력: "스포츠"
        출력: 잠실 야구장, 서울 월드컵 경기장 ...

        {format_instructions}""",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)
pprint(prompt.partial_variables)

{'format_instructions': 'Your response should be a list of comma separated '
                        'values, eg: `foo, bar, baz` or `foo,bar,baz`'}


In [67]:
# 모델 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.2
)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

# "음식"에 대한 체인 호출 실행
result = chain.invoke({"subject": "음식"})

# 쉼표로 구분된 리스트 출력
print("음식 관련 목록:")
print(result)

음식 관련 목록:
['종로 비빔밥', '광화문 닭볶음탕', '부산 개미집', '전주 한옥마을 순두부찌개', '대구 서문시장 치킨']


In [ ]:
# "영화"에 대한 체인 호출 실행
result = chain.invoke({"subject": "영화"})

# 쉼표로 구분된 리스트 출력
print("영화 관련 목록:")
print(result)

영화 관련 목록:
['부산 영화의 전당', 'CGV 홍대', '롯데시네마 월드몰', '한국 영화 박물관', '충무로 영화 거리']


In [ ]:
# "스포츠"에 대한 체인 호출 실행
result = chain.invoke({"subject": "스포츠츠"})

# 쉼표로 구분된 리스트 출력
print("스포츠 관련 목록:")
print(result)

스포츠 관련 목록:
['잠실 야구장', '서울 월드컵 경기장', '부산 광안리 해수욕장(서핑)', '강원도 평창 스키장', '인천 영종도 골프장']


##### 문제 2-2 : 영화 리뷰 감정 분석기
* 영화 리뷰 텍스트를 입력받아 감정을 "긍정", "부정", "보통" 중 하나로 분류하는 시스템을 만드세요.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint

# 감정 클래스 정의 (Enum)
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "중립"

# EnumOutputParser 초기화
enumParser = EnumOutputParser(enum=Sentiment)
format_instructions = enumParser.get_format_instructions()

print("감정 분류 출력 형식:")
print(format_instructions)

# 프롬프트 템플릿
template = """
당신은 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)   
print(prompt)

In [ ]:
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0  # 일관성을 위해 0으로 설정
)

# OutputFixingParser로 안정성 향상
fixing_parser = OutputFixingParser.from_llm(parser=enumParser, llm=model)

print("모델 및 파서 설정 완료")

# 테스트 텍스트
texts = [
    "이 제품 정말 좋아요! 완전 만족스러워요.",
    "서비스가 너무 느리고 불친절했습니다.",
    "오늘은 비가 온다네요.",
    "배송은 빠르지만 품질이 아쉽습니다.",
    "최고의 경험이었습니다!",
    "완전 실망했어요... 최악이에요"
]

print(f"테스트할 텍스트 {len(texts)}개 준비 완료")

In [ ]:
# 안전한 감정 분석 함수 (에러 처리 포함)
def safe_sentiment_analysis(text, use_fixing_parser=True):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        # 기본 체인 생성
        chain = prompt | model | (fixing_parser if use_fixing_parser else parser)
        
        # 분석 실행
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

# 실제 감정 분석 실행 (API 키 필요)
def run_sentiment_analysis():
    """실제 감정 분석 실행"""
    print("=== 실제 감정 분석 결과 ===")
    
    success_count = 0
    total_count = len(texts)
    
    for i, text in enumerate(texts, 1):
        print(f"\n{i}. 텍스트: {text}")
        
        # OutputFixingParser 사용
        result, error = safe_sentiment_analysis(text, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            # 기본 파서로 재시도
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
    print(f"\n=== 결과 요약 ===")
    print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"실패: {total_count-success_count}/{total_count}")

# 실제 분석 실행 (API 키가 있는 경우)
try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")

##### 문제 2-3: 학생 정보 구조화 시스템
* 학생의 자유 형식 자기소개를 입력받아 이름, 나이, 전공, 취미 리스트, 목표를 구조화된 형태로 추출하는 시스템을 만드세요.

##### 문제 2-4 : 여행 계획 분석기
* 여행 후기나 계획 텍스트를 입력받아 여행지, 기간, 예산, 추천도(1-5점), 주요 활동 리스트를 구조화된 형태로 추출하는 시스템을 만드세요.